# Note book for the model architecture and training part

## 1. Loading the necessary libraries

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2 as cv
import os
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
from keras.utils import plot_model
from keras.models import load_model
from sklearn.model_selection import train_test_split
from PIL import Image
import time


## 2. Configuring TensorFlow: GPU Growth and Soft Placement

In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True

## 3. Loadind and balance the data

### 3.1. Loading the data

In [ ]:
# Load the data
df = pd.read_csv('../frameRecorder/databis.csv')

df.head()

### 3.2. Balancing the data

In [ ]:
counts = df.sum()
print(counts)

df = df.drop(df[df.z == 1].sample(frac=0.6).index)
df = df.drop(df[df.d == 1].sample(frac=0.04).index)
df = df.drop(df[df.zq == 1].sample(frac=0.4).index)
df = df.drop(df[df.zd == 1].sample(frac=0.4).index)

new_counts = df.sum()
print(new_counts)

In [ ]:
X, y = df.iloc[:, 0], df.iloc[:, 1:]
print(X[0:10])
print(X.shape)
print(y.shape)

### Extra: Exemple of the preprocessing we perform on a road image

In [ ]:
img = cv.imread('../frameRecorder/screenshots/1682028946060.png')
# make it one channel
img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
img = cv.Canny(img, threshold1=120, threshold2=220)
img = img[100: , :]

lanes = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
lanes[:, :, 0] = img
lanes[:, :, 1] = img
lanes[:, :, 2] = img


# get the lines from lanes image 
lines = cv.HoughLinesP(img, rho=1, theta=np.pi/90, threshold=70, minLineLength=20, maxLineGap=50)

# draw the lines on the lanes image
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv.line(lanes, (x1, y1), (x2, y2), (0, 255, 0), 5)

lanes = cv.resize(lanes, (200, 66))

# show the image
plt.imshow(lanes)
plt.show()

## 4. Data Extraction: Road and Map Analysis

In [ ]:
XX_road = []
XX_map = []
progress = 0
total = len(X)
for i in X:
    img = cv.imread('../frameRecorder/screenShots/' + str(i) + '.png')

    # part to get the data about the road
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.Canny(img, threshold1=120, threshold2=220)
    img = img[100: , :]

    lanes = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    lanes[:, :, 0] = img
    lanes[:, :, 1] = img
    lanes[:, :, 2] = img


    # get the lines from lanes image 
    lines = cv.HoughLinesP(img, rho=1, theta=np.pi/90, threshold=70, minLineLength=20, maxLineGap=50)

    # draw the lines on the lanes image
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv.line(lanes, (x1, y1), (x2, y2), (0, 255, 0), 5)

    lanes = cv.resize(lanes, (200, 66))
    
    lanes = lanes / 255
    XX_road.append(lanes)
    # end of part to get the data about the road
    
    # part to get the data about the map
    img = cv.imread('../frameRecorder/screenShots/' + str(i) + '_map.png')
    img = img / 255
    XX_map.append(img)
    # end of part to get the data about the map

    progress += 1
    print(str(progress) + '/' + str(total))
    os.system('cls')

## 5. Spliting the data into Train, Test and Validation datasets

In [ ]:
# split the data into training , validation and testing sets
X_train_road, X_test_road, X_train_map, X_test_map, y_train, y_test = train_test_split(XX_road, XX_map, y, test_size=0.2, random_state=42)
X_test_road, X_val_road, X_test_map, X_val_map, y_test, y_val = train_test_split(X_test_road, X_test_map, y_test, test_size=0.5, random_state=42)

In [ ]:
X_train_road = np.array(X_train_road)
X_test_road = np.array(X_test_road)
X_val_road = np.array(X_val_road)
X_train_map = np.array(X_train_map)
X_test_map = np.array(X_test_map)
X_val_map = np.array(X_val_map)

In [ ]:
print(X_train_road.shape)
print(X_test_road.shape)
print(X_val_road.shape)
print(X_train_map.shape)
print(X_test_map.shape)
print(X_val_map.shape)

## 6. Creating the model

### 6.1. Creating the model structure

In [ ]:
with tf.compat.v1.Session(config=config) as sess:
    model_1 = Sequential([
        Conv2D(24, (5, 5), (2, 2), activation='relu', input_shape=(66, 200, 3)),
        Conv2D(36, (5, 5), (2, 2), activation='relu'),
        Conv2D(48, (5, 5), (2, 2), activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Dense(100, activation='relu'),
        Dense(50, activation='relu'),
        Dense(10, activation='relu'),
    ])
    # another model 
    model_2 = Sequential([
        Conv2D(22, kernel_size=(3, 3), activation='relu', input_shape=(100, 145, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(28, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(34, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),

        Conv2D(40, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        Flatten(),
        Dense(100, activation='relu'),
        Dense(50, activation='relu'),
        Dense(15, activation='relu'),
    ])

    # concatenate two models
    combinedInput = layers.concatenate([model_1.output, model_2.output])
    x = Dense(8, activation="softmax")(combinedInput)
    model = keras.Model(inputs=[model_1.input, model_2.input], outputs=x)
    

### 6.2. Compiling the model

In [ ]:
# compile the model
with tf.compat.v1.Session(config=config) as sess:
    # compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize the model
    model.summary()

### 6.3. Saving a graph representation of the model structure

In [ ]:
# print a graph of the model
with tf.compat.v1.Session(config=config) as sess:
    tf.keras.utils.plot_model(model, "model.png", show_shapes=True)

In [ ]:
# show the image model.png 
plot_model(model, show_shapes=True, show_layer_names=True)

### 6.4. Training the model and saving the best model found at each epoch

In [ ]:
# train the model
with tf.compat.v1.Session(config=config) as sess:
    # select the best model
    checkpoint = ModelCheckpoint('final_model_v1_9_0.h5', monitor='val_accuracy', verbose=0, save_best_only=True, mode='auto')
    # train the model
    history = model.fit([X_train_road, X_train_map], y_train, epochs=25, batch_size=32, validation_data=([X_val_road, X_val_map], y_val), callbacks=[checkpoint])

### 6.5. Testing the model and plotting the result

In [ ]:
# evaluate the model
with tf.compat.v1.Session(config=config) as sess:
    # load the model
    model = load_model('final_model_v1_9_0.h5') 

    loss, acc = model.evaluate([X_test_road, X_test_map], y_test, verbose=2)
    print('Test Accuracy: {}'.format(acc))
    print('Test Loss: {}'.format(loss))

    # plot the accuracy
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0.5, 1])
    plt.legend(loc='lower right')
    plt.show()

    # plot the loss
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.ylim([0, 1])
    plt.legend(loc='lower right')
    plt.show()

# The end :)